In [2]:
import pandas as pd
import os

In [ ]:
# 데이터 병합 코드

# CSV 파일이 저장된 폴더 경로
folder_path = "C:/Users/wnstj/OneDrive/바탕 화면/snsa"

# 식당 데이터만 필터링 (파일 이름에 'restaurant'가 포함된 파일만)
csv_files = [file for file in os.listdir(folder_path) if 'restaurant' in file and file.endswith('post.csv')]

# 데이터프레임 리스트 생성
df_list = []

for file in csv_files:
    file_path = os.path.join(folder_path, file)  # 파일 경로
    df = pd.read_csv(file_path, encoding='cp949')  # CSV 파일 읽기
    
    # 열 이름 확인 및 데이터 검증
    required_columns = ['restaurant name', 'address', 'content']  # 주요 열 이름
    if all(col in df.columns for col in required_columns):  # 필수 열이 모두 있는 경우만 추가
        df_list.append(df)
    else:
        print(f"파일 {file}은(는) 필수 열이 없어 제외됩니다.")

# 모든 데이터프레임을 하나로 합치기

if df_list:
    merged_df = pd.concat(df_list, ignore_index=True)
    # 합쳐진 데이터 저장
    merged_df.to_csv('post.csv', encoding='cp949', index=False)
    print("식당 데이터가 성공적으로 합쳐졌습니다!")
else:
    print("조건에 맞는 파일이 없습니다.")

In [ ]:
# 크롤링 과정에서 필수 요소가 추출되지 않은 데이터 삭제

rest_df = pd.read_csv('post.csv', encoding='cp949') # 위 코드로 병합된 csv와 같은 csv파일

display(rest_df)

# NaN 처리 함수 정의
def clean_text(text):
    # 대괄호([])와 작은따옴표(') 제거하는 함수
    if isinstance(text, str):  # 문자열일 때만 처리
        return text.strip("[]").replace("'", "").strip()
    return text  # 문자열이 아니면 그대로 반환

# content가 NaN인 경우 처리 (예: 특정 값으로 대체하거나 제거)
rest_df['content'] = rest_df['content'].fillna('')  # NaN을 빈 문자열로 대체

# 정리할 필드 리스트
fields_to_clean = ['address', 'comments', 'hashtags', 'restaurant name', 'content']

# 각 필드 정리
for field in fields_to_clean:
    if field in rest_df.columns:
        rest_df[field] = rest_df[field].fillna('').apply(clean_text)

rest_df = rest_df[(rest_df['restaurant name'] != '없음') & (rest_df['address'] != '없음')]

print("모든 NaN 및 텍스트 정리 완료!")

In [ ]:
# 광고 단어 리스트
ad_text_list = ['부업', '광고', '협찬', '재테크', '출금', '공짜', '수익', '카톡', '원금', '댓글',
                '할인', '이벤트', '부자', 'Repost', '구매', '문의전화', '오시는']

# content 필드에서 광고 단어가 포함된 경우 광고로 판단
rest_df['is_ad'] = rest_df['content'].apply(
    lambda content: any(block_word in content for block_word in ad_text_list)
).astype(int)  # 내용에 광고 단어가 있으면 1(광고), 없으면 0(비광고)로 분류류

rest_df

In [ ]:
from contents_DB import connect_mongodb, save_to_mongodb_unique

# MongoDB 연결
collection = connect_mongodb('post','all_post')

In [ ]:
# MongoDB에 전처리된 데이터 저장
save_to_mongodb_unique(rest_df, collection)
print("Data saved to MongoDB successfully!")